In [26]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from shapely import wkt
import re
import spacy
import csv
import random
from spacy.lang.en.examples import sentences 
from geopy.distance import great_circle
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import string
from difflib import SequenceMatcher

In [2]:
#Small analysis and processing task 
njglobe_1 = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/newjerseyglobe_locations_1.csv')
print(njglobe_1.columns)
print(njglobe_1.shape)
#print(len(fl))

Index(['Location', 'Latitude', 'Longitude', 'Count'], dtype='object')
(1123, 4)


In [4]:
njglobe_1 = njglobe_1.dropna(subset=["Latitude", "Longitude"])
print(njglobe_1.shape)
#So there are 23 rows with missing data

(1100, 4)


In [6]:
import folium

map_center = [njglobe_1["Latitude"].mean(), njglobe_1["Longitude"].mean()]
map_zoom = 10
mymap = folium.Map(location=map_center, zoom_start=map_zoom)

# Pick a specific color (e.g., red)
color = "#FF0000"

# Add circle markers for each location (dots) with the chosen color and labels
for index, row in njglobe_1.iterrows():
    location = row["Location"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    popup_text = f"Location: {location}"
    folium.CircleMarker([latitude, longitude], popup=popup_text, radius=5, color=color, fill=True, fill_color=color).add_to(mymap)

# Saving the map
mymap.save("trial_2.html")

In [7]:
map_center = [njglobe_1["Latitude"].mean(), njglobe_1["Longitude"].mean()]
map_zoom = 10
mymap = folium.Map(location=map_center, zoom_start=map_zoom)

# Find the maximum count to use for scaling the marker size
max_count = njglobe_1["Count"].max()

# Function to map count values to marker sizes (adjust the scale factor as needed)
def get_marker_size(count):
    scale_factor = 10  # Adjust this value to control the marker size scaling
    return 5 + (count / max_count) * scale_factor

# Add circle markers for each location with varying size based on the count and labels
for index, row in njglobe_1.iterrows():
    location = row["Location"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    count = row["Count"]
    popup_text = f"Location: {location}\nCount: {count}"
    marker_size = get_marker_size(count)
    folium.CircleMarker([latitude, longitude], popup=popup_text, radius=marker_size, color="#FF0000", fill=True, fill_color="#FF0000").add_to(mymap)

# Saving the map
mymap.save("trial_3.html")

In [15]:
# Calculate the map center and zoom level as before
map_center = [njglobe_1["Latitude"].mean(), njglobe_1["Longitude"].mean()]
map_zoom = 5
mymap = folium.Map(location=map_center, zoom_start=map_zoom)

# Set a single color for all the markers
color = "#FF0000"  # Red color

# Define a function to map count values to marker sizes
def get_marker_size(count):
    scale_factor = 40  # Adjust this value to control the marker size scaling
    return 5 + (count / max_count) * scale_factor

# Find the maximum count to use for scaling the marker size
max_count = njglobe_1["Count"].max()

# Add circle markers with varying size based on the count
for _, row in njglobe_1.iterrows():
    location = row["Location"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    count = row["Count"]
    marker_size = get_marker_size(count)
    popup_text = f"Location: {location}\nCount: {count}"
    folium.CircleMarker([latitude, longitude], popup=popup_text, radius=marker_size, color=color, fill=True, fill_color=color).add_to(mymap)

# Saving the map
mymap.save("scatter_plot_visualization_3.html")


In [36]:
six_abc = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/news_articles_info_6abc.com.csv')
six_abc.shape

(235439, 9)

In [37]:
six_abc = six_abc.dropna(subset=["gpe_latitude", "gpe_longitude"])
print(six_abc.shape)
#no null data

(235439, 9)


#Grouping the location names based on the same latitude and longitude values

In [38]:
# Group the data based on 'gpe_latitude' and 'gpe_longitude' and create a new column 'group_ID'
six_abc['group_ID'] = six_abc.groupby(['gpe_latitude', 'gpe_longitude']).ngroup()

# Sort the DataFrame based on the 'group_ID' column
six_abc.sort_values(by='group_ID', inplace=True)

# Reset the index after sorting (optional)
six_abc.reset_index(drop=True, inplace=True)

# Save the modified DataFrame to a new CSV file
six_abc.to_csv('grouped_data_six_abc.csv', index=False)

print("Data saved to 'grouped_data_six_abc.csv'.")


Data saved to 'grouped_data_six_abc.csv'.


In [39]:
#Small analysis and processing task 
df = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/grouped_data_six_abc.csv')
print(df.columns)
print(df.shape)
#print(len(fl))

Index(['news_url', 'outlet', 'outlet_latitude', 'outlet_longitude', 'gpe',
       'gpe_latitude', 'gpe_longitude', 'gpe_county', 'gpe_occurrences',
       'group_ID'],
      dtype='object')
(235439, 10)


#Rename the gpe based on the common string for the same group_ID

In [42]:
grouped_df = df.groupby('group_ID')['gpe'].apply(list).reset_index()

# Function to find the common pattern among a list of strings
def find_common_pattern(strings):
    if len(strings) == 0:
        return None

    # Initialize SequenceMatcher with the first string
    seq_matcher = SequenceMatcher(None, strings[0], "")

    # Iterate through the rest of the strings and find the longest common substring
    common_substring = strings[0]
    for s in strings[1:]:
        seq_matcher.set_seq2(s)
        match = seq_matcher.find_longest_match(0, len(strings[0]), 0, len(s))
        if match.size > 0:
            common_substring = strings[0][match.a: match.a + match.size]
            break

    # Check if the pattern is valid and appears in all strings
    if common_substring and all(common_substring in s for s in strings):
        return common_substring
    else:
        return None

# Find the common pattern for each group and rename the location names accordingly
for _, group in grouped_df.iterrows():
    group_id = group['group_ID']
    locations = group['gpe']

    common_pattern = find_common_pattern(locations)

    if common_pattern:
        # Update the location names with the common pattern
        df.loc[df['group_ID'] == group_id, 'gpe'] = common_pattern

# Save the modified dataframe to a new CSV file
df.to_csv('modified_location_data_six_abc.csv', index=False)

In [43]:
df.shape

(235439, 10)

#Delete duplicate values for each location

In [54]:
df = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/modified_location_data_six_abc.csv')
# df.drop_duplicates(subset='gpe', keep='first', inplace=True)
# df.shape

In [55]:
# Group by 'gpe' and calculate the sum of 'gpe_occurrences'
grouped_df = df.groupby('gpe', as_index=False)['gpe_occurrences'].sum()

# Merge the original DataFrame with the grouped DataFrame on 'gpe'
merged_df = df.merge(grouped_df, on='gpe', how='left', suffixes=('', '_sum'))

# Update 'gpe_occurrences' column with the calculated sum for the first occurrence
merged_df.loc[merged_df.duplicated('gpe'), 'gpe_occurrences'] = merged_df['gpe_occurrences_sum']

# Drop unnecessary columns
merged_df.drop(['gpe_occurrences_sum'], axis=1, inplace=True)

# Save the modified DataFrame to a new CSV file
merged_df.to_csv('merged_data_sixabc.csv', index=False)

In [56]:
# Group by 'gpe' and calculate the sum of 'gpe_occurrences'
grouped_df = df.groupby('gpe', as_index=False)['gpe_occurrences'].sum()

# Merge the original DataFrame with the grouped DataFrame on 'gpe'
merged_df = df.merge(grouped_df, on='gpe', how='left', suffixes=('', '_sum'))

# Update 'gpe_occurrences' column with the calculated sum
merged_df['gpe_occurrences'] = merged_df['gpe_occurrences_sum']

# Drop the unnecessary column 'gpe_occurrences_sum'
merged_df.drop(['gpe_occurrences_sum'], axis=1, inplace=True)

# Save the modified DataFrame to a new CSV file
merged_df.to_csv('merged_data_sixabc_1.csv', index=False)

print("Data saved to 'merged_data_sixabc_1.csv'.")

Data saved to 'merged_data_sixabc_1.csv'.


In [57]:
merged_df.shape

(235439, 10)

In [58]:
remove_duplicates_df = merged_df.drop_duplicates(subset='gpe', keep='first')
remove_duplicates_df.shape

(2108, 10)

In [59]:
remove_duplicates_df.to_csv('no_duplicates_found_sixabc.csv', index=False)

#Verify that all the GPE's are verified location, if not drop them

In [60]:
df = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/no_duplicates_found_sixabc.csv')

In [62]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from nltk.tokenize import word_tokenize
from nltk import pos_tag

In [63]:
def is_location_with_geonames(word):
    tokens = word_tokenize(word)
    tagged_words = pos_tag(tokens)
    is_proper_noun = any(pos in ['NNP', 'NNPS'] for _, pos in tagged_words)
    
    if not is_proper_noun:
        return False

    # Check if the word is a location using Geonames
    geolocator = Nominatim(user_agent="location_checker",timeout=8)
    try:
        location = geolocator.geocode(word)
        if location is not None:
            return True
        else:
            return False
    except GeocoderTimedOut:
        return is_location_with_geonames(word)  # Retry the request if a timeout occurs
    
# Create a boolean mask to indicate whether each row is a valid location or not
valid_locations_mask = df['gpe'].apply(is_location_with_geonames)

# Keep only the rows with valid locations and drop the rest
df = df[valid_locations_mask]

# Save the modified dataframe to a new CSV file
df.to_csv('valid_gpes_sixabc.csv', index=False)

In [64]:
df.shape

(1190, 10)

In [65]:
sixabc_valid = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/valid_gpes_sixabc.csv')

In [72]:
sixabc_valid_1 = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/valid_gpes_sixabc_1.csv')

#Visulaising these gpe points on the map

In [73]:
# Calculate the map center and zoom level as before
map_center = [sixabc_valid_1["gpe_latitude"].mean(), sixabc_valid_1["gpe_longitude"].mean()]
map_zoom = 5
mymap = folium.Map(location=map_center, zoom_start=map_zoom)

# Set a single color for all the markers
color = "#2F539B" 

# Define a function to map count values to marker sizes
def get_marker_size(count):
    scale_factor = 40  # Adjust this value to control the marker size scaling
    return 5 + (count / max_count) * scale_factor

# Find the maximum count to use for scaling the marker size
max_count = sixabc_valid_1["gpe_occurrences"].max()

# Add circle markers with varying size based on the count
for _, row in sixabc_valid_1.iterrows():
    location = row["gpe"]
    latitude = row["gpe_latitude"]
    longitude = row["gpe_longitude"]
    count = row["gpe_occurrences"]
    marker_size = get_marker_size(count)
    popup_text = f"Location: {location}\nCount: {count}"
    folium.CircleMarker([latitude, longitude], popup=popup_text, radius=marker_size, color=color, fill=True, fill_color=color).add_to(mymap)

# Saving the map
mymap.save("scatter_plot_visualization_sixabc_1.html")

#Generating a heatmap

In [74]:
# Create a map centered at the mean latitude and longitude of the data points
center_latitude = sixabc_valid_1['gpe_latitude'].mean()
center_longitude = sixabc_valid_1['gpe_longitude'].mean()
heatmap_map = folium.Map(location=[center_latitude, center_longitude], zoom_start=2)

# Create a list of tuples containing the Latitude and Longitude for each data point
heat_data = list(zip(sixabc_valid_1['gpe_latitude'], sixabc_valid_1['gpe_longitude'], sixabc_valid_1['gpe_occurrences']))

# Add the HeatMap layer to the map
HeatMap(heat_data).add_to(heatmap_map)

# Display the heatmap
heatmap_map.save('heatmap_map_sixabc_1.html')

In [75]:
# Create a map centered at the mean latitude and longitude of the data points
center_latitude = sixabc_valid_1['gpe_latitude'].mean()
center_longitude = sixabc_valid_1['gpe_longitude'].mean()
heatmap_map = folium.Map(location=[center_latitude, center_longitude], zoom_start=10, tiles='Stamen Terrain')

# Create a list of tuples containing the Latitude, Longitude, and Occurrences for each data point
heat_data = list(zip(sixabc_valid_1['gpe_latitude'], sixabc_valid_1['gpe_longitude'], sixabc_valid_1['gpe_occurrences']))

# Add the HeatMap layer to the map with custom gradient and opacity
HeatMap(heat_data, gradient={0.4: 'blue', 0.65: 'green', 1: 'red'}, opacity=0.7, radius=15, blur=20).add_to(heatmap_map)

# Display the heatmap
heatmap_map.save('heatmap_map_sixabc_tuned_1.html')

#Applying the logarithmic transformations

In [77]:
sixabc_valid = pd.read_csv('/Users/gayathri/Documents/nj-local-news-analysis/valid_gpes_sixabc.csv')

In [78]:
# Apply log transformation to the 'gpe_occurrences' column and create a new column
sixabc_valid['log_gpe_occurrences'] = np.log(sixabc_valid['gpe_occurrences'])

# Save the modified DataFrame to a new CSV file
sixabc_valid.to_csv('map_view_sixabc.csv', index=False)
sixabc_valid.head

<bound method NDFrame.head of                                                news_url    outlet  \
0     https://6abc.com/entertainment/metallica-delay...  6abc.com   
1     https://6abc.com/sports/new-mls-playoff-format...  6abc.com   
2     https://6abc.com/society/meghan-and-harry-phot...  6abc.com   
3                             https://6abc.com/5840218/  6abc.com   
4                         https://6abc.com/tag/inmates/  6abc.com   
...                                                 ...       ...   
1185                          https://6abc.com/5839122/  6abc.com   
1186                          https://6abc.com/5643306/  6abc.com   
1187  https://6abc.com/business/chick-fil-a-to-close...  6abc.com   
1188  https://6abc.com/brexit-hangs-in-the-balance-a...  6abc.com   
1189  https://6abc.com/food/6-great-spots-for-ice-cr...  6abc.com   

      outlet_latitude  outlet_longitude                   gpe  gpe_latitude  \
0           41.128848        -74.686881           New Zealand 

In [81]:
# Calculate the map center and zoom level as before
map_center = [sixabc_valid["gpe_latitude"].mean(), sixabc_valid["gpe_longitude"].mean()]
map_zoom = 5
mymap = folium.Map(location=map_center, zoom_start=map_zoom)

# Set a single color for all the markers
color = "#2F539B" 

# Define a function to map count values to marker sizes
def get_marker_size(count):
    scale_factor = 40  # Adjust this value to control the marker size scaling
    return 5 + (count / max_count) * scale_factor

# Find the maximum count to use for scaling the marker size
max_count = sixabc_valid["log_gpe_occurrences"].max()

# Add circle markers with varying size based on the count
for _, row in sixabc_valid.iterrows():
    location = row["gpe"]
    latitude = row["gpe_latitude"]
    longitude = row["gpe_longitude"]
    count = row["log_gpe_occurrences"]
    marker_size = get_marker_size(count)
    popup_text = f"Location: {location}\nCount: {count}"
    folium.CircleMarker([latitude, longitude], popup=popup_text, radius=marker_size, color=color, fill=True, fill_color=color).add_to(mymap)

# Saving the map
mymap.save("scatter_plot_visualization_sixabc_2.html")

In [84]:
# Create a map centered at the mean latitude and longitude of the data points
center_latitude = sixabc_valid['gpe_latitude'].mean()
center_longitude = sixabc_valid['gpe_longitude'].mean()
heatmap_map = folium.Map(location=[center_latitude, center_longitude], zoom_start=10, tiles='Stamen Terrain')

# Create a list of tuples containing the Latitude, Longitude, and Occurrences for each data point
heat_data = list(zip(sixabc_valid['gpe_latitude'], sixabc_valid['gpe_longitude'], sixabc_valid['log_gpe_occurrences']))

# Add the HeatMap layer to the map with custom gradient and opacity
HeatMap(heat_data, gradient={0.4: 'blue', 0.65: 'green', 1: 'red'}, opacity=0.7, radius=15, blur=20).add_to(heatmap_map)

# Display the heatmap
heatmap_map.save('heatmap_map_sixabc_tuned_2.html')